# Import libraries and info from DB

In [1]:
## Import libraries
import requests
import pandas as pd
import pymysql
from sqlalchemy import create_engine

In [2]:
## Connect to MySQL DB - LOCAL   
driver   = 'mysql+pymysql:'
user     = 'root' ## the user you use to access the DB
password = 'ML_LI_04' ## the password you use to access the DB
ip       = '127.0.0.1' ## the DB host IP
database = 'data_thieves_dummy' ## name_of_DB_you_want_to_connect

# ## Connect to MySQL DB - CLOUD   
# driver   = 'mysql+pymysql:'
# user     = 'root' ## the user you use to access the DB
# password = 'ML_LI_04' ## the password you use to access the DB
# ip       = '35.195.172.42' ## the DB host IP
# database = 'data_thieves' ## name_of_DB_you_want_to_connect

connection_string = f'{driver}//{user}:{password}@{ip}/{database}'
engine = create_engine(connection_string)

In [3]:
tracks = pd.read_sql('track', engine)

C:\Users\miner\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1366, "Incorrect string value: '\\xE1ndar ...' for column 'VARIABLE_VALUE' at row 1")
  result = self._query(query)


In [4]:
artists = pd.read_sql('artist', engine)

In [5]:
top_50_country = pd.read_sql('top_artists_country', engine)


# Check music profile per country

Merge columns in one dataframe:

In [6]:
artists.rename(columns = {'ID': 'artist_id'}, inplace = True)

In [7]:
tracks_with_artist = pd.merge(tracks, artists, on='artist_id')
tracks_with_artist.rename(columns = {'ID': 'track_id'}, inplace = True)
tracks_with_artist.head(3)

,track_id,track_name,spotify_id,acousticness,danceability,duration_ms,energy,instrumentalness,track_key,liveness,loudness,track_mode,speechiness,tempo,time_signature,valence,popularity,artist_id,artist_name
0,1,C'est beau de faire un Show,0BRjO6ga9RKCKjfDqeFgWV,0.611,0.389,99373,0.9100,0.0,C#,0.3460,-1.828,Major,0.0525,166.969,4/4,0.814,0,1,Henri Salvador
1,2,Dis-moi Monsieur Gordon Cooper,0Gc6TVm52BwZD07Ki6tIvf,0.703,0.240,152427,0.3260,0.0,C#,0.0985,-12.178,Major,0.0395,171.758,4/4,0.227,0,1,Henri Salvador
2,3,Le petit souper aux chandelles,0Mf1jKa8eNAf1a4PwTbizj,0.749,0.578,160627,0.0948,0.0,C#,0.1070,-14.970,Major,0.1430,87.479,4/4,0.358,0,1,Henri Salvador


In [8]:
all_tracks_top_50 = pd.merge(tracks_with_artist, top_50_country, on='artist_id')
all_tracks_top_50.head(3)

,track_id,track_name,spotify_id,acousticness,danceability,duration_ms,energy,instrumentalness,track_key,liveness,...,time_signature,valence,popularity,artist_id,artist_name_x,ID,ranking,artist_name_y,country_name,country_id
0,17481,Desperado,6KFaHC9G178beAp7P0Vi5S,0.323,0.685,186467,0.61,0.0,C,0.102,...,3/4,0.323,63,37,Rihanna,47,17,Rihanna,Jordan,112
1,17481,Desperado,6KFaHC9G178beAp7P0Vi5S,0.323,0.685,186467,0.61,0.0,C,0.102,...,3/4,0.323,63,37,Rihanna,48,21,Rihanna,Switzerland,213
2,17481,Desperado,6KFaHC9G178beAp7P0Vi5S,0.323,0.685,186467,0.61,0.0,C,0.102,...,3/4,0.323,63,37,Rihanna,49,22,Rihanna,Spain,206


In [9]:
all_tracks = all_tracks_top_50.drop(['spotify_id', 'artist_name_x', 'ID', 'artist_name_y'], axis = 1, inplace = True)

In [10]:
#all_tracks.to_csv('all_tracks.csv', sep=";")


Calculate mean for each 50 songs per country on the different values

In [11]:
jordan_tracks = all_tracks[(all_tracks.country_name == 'Jordan')]
jordan_means = jordan_tracks.mean(axis = 0) 

AttributeError: 'NoneType' object has no attribute 'country_name'

In [ ]:
switzerland_tracks = all_tracks[(all_tracks.country_name == 'Switzerland')]
switzerland_means = switzerland_tracks.mean(axis = 0) 

In [ ]:
spain_tracks = all_tracks[(all_tracks.country_name == 'Spain')]
spain_means = spain_tracks.mean(axis = 0) 


Create compare table

In [ ]:
compare_means = pd.concat([spain_means, switzerland_means, jordan_means], axis = 1)
compare_means.columns = ['Spain', 'Switzerland', 'Jordan']

In [ ]:
compare_means.head(3)

In [ ]:
all_tracks[(all_tracks['ranking'] == 28)].groupby('artist_id').count()

# Check possible tendencies in music styles in top 50 (per country)

### Jordan

In [ ]:
jordan_tracks_2 = all_tracks[(all_tracks.country_name == 'Jordan')]
jordan_tracks_ordered = jordan_tracks_2.sort_values(by=['ranking'])
jordan_tracks_by_artist = jordan_tracks_ordered.groupby(['ranking', 'artist_name']).mean().copy()
jordan_tracks_by_artist.drop('artist_id', axis = 1, inplace = True)
jordan_tracks_by_artist

### Switzerland

In [ ]:
switzerland_tracks_2 = all_tracks[(all_tracks.country_name == 'Switzerland')]
switzerland_tracks_ordered = switzerland_tracks_2.sort_values(by=['ranking'])
switzerland_tracks_by_artist = switzerland_tracks_ordered.groupby(['ranking', 'artist_name']).mean().copy()
switzerland_tracks_by_artist.drop('artist_id', axis = 1, inplace = True)
switzerland_tracks_by_artist

In [ ]:
# sw_for_plot = switzerland_tracks_by_artist.drop('duration_ms', axis = 1, inplace = True)
switzerland_tracks_by_artist.plot()

### Spain

In [ ]:
spain_tracks_2 = all_tracks[(all_tracks.country_name == 'Spain')]
spain_tracks_ordered = spain_tracks_2.sort_values(by=['ranking'])
spain_tracks_by_artist = spain_tracks_ordered.groupby(['ranking', 'artist_name']).mean().copy()
spain_tracks_by_artist.drop('artist_id', axis = 1, inplace = True)
spain_tracks_by_artist

In [ ]:
compare_means_clean = compare_means.drop(['duration_ms','artist_id', 'ranking'], axis = 0).copy()

In [ ]:
compare_means_clean.plot()

In [ ]:
spain_tracks_by_artist.to_csv('spain_top_50.csv')